In [ ]:
# Import necessary libraries
import warnings
from vectorbtpro import *

warnings.filterwarnings("ignore")
vbt.settings.set_theme('dark')
vbt.settings.plotting.use_resampler=(True)
vbt.settings['plotting']['layout']['width']=600
vbt.settings['plotting']['layout']['height']=200



# Import data


In [115]:
data = vbt.YFData.pull(['AAPL','MSFT','GOOGL','AMZN','META'], start= '2015-01-01', end= '2024-01-01')
spy = vbt.YFData.pull('SPY', start= '2015-01-01', end= '2024-01-01')

## Create SMA Cross Strategy

In [248]:
close = data.get('Close')
sma1 = data.run('sma', timeperiod=10)
sma2 = data.run('sma', timeperiod=200)
# The sma_timeperiod is a multiindex, so we need to drop it
sma1_cleaned = sma1.sma.droplevel('sma_timeperiod', axis=1)
sma2_cleaned = sma2.sma.droplevel('sma_timeperiod', axis=1)

# print(sma1_cleaned.tail())
# Set up your buy and sell signals
entries = (sma1_cleaned > sma2_cleaned)
exits = (sma1_cleaned < sma2_cleaned)

# Create multiple simulations one for each symbol
pf_sma = vbt.Portfolio.from_signals(close, entries, exits, init_cash=1000)
pf_buy_and_hold = vbt.Portfolio.from_holding(close, init_cash=1000)

# Create RSI mean reversion strategy
rsi = data.run('rsi', window=14)
# The RSI is a multiindex, so we need to drop it
rsi_cleaned = rsi.rsi.droplevel('rsi_window', axis=1)

# Set up your buy and sell signals
entries = (rsi_cleaned < 30)
exits = (rsi_cleaned > 70)
# Create multiple simulations one for each symbol
pf_rsi = vbt.Portfolio.from_signals(close, entries, exits, init_cash=1000)
pf_buy_and_hold = vbt.Portfolio.from_holding(close, init_cash=1000)

# Show all total returns for each strategy
pd.concat([pf_sma.total_return, pf_rsi.total_return, pf_buy_and_hold.total_return], axis=1, keys=['SMA', 'RSI', 'Buy and Hold'])
# Just grab a single simulation
# pf['AAPL'].stats()

,SMA,RSI,Buy and Hold
symbol,,,
AAPL,4.606626,2.557465,6.869142
MSFT,5.605157,0.960391,8.295786
GOOGL,1.745760,1.034712,4.275800
AMZN,2.986390,1.058740,8.849605
META,2.793333,0.382338,3.511919


# Now let's build a portfolio of these strategies
For this demo I'll rebalance monthly based on equal weight and an alternate portfolio sim using a slightly different weighting strategy. Note later in the notebook we will create a rebalancing strategy based on regimes and a regime filter. However for learning purposes we will go one step at a time.

In [246]:
# Here we will treat each sim as a different asset
sma_strategy = pf_sma.value
rsi_strategy = pf_rsi.value


# Add _sma and _rsi to the end of each strategy
sma_strategy.columns = sma_strategy.columns + '_sma'
rsi_strategy.columns = rsi_strategy.columns + '_rsi'

# Single dataframe
strategy = pd.concat([sma_strategy, rsi_strategy], axis=1)



In [254]:
symbol_wrapper.columns

Index(['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META'], dtype='object', name='symbol')

In [256]:

# Let's rebalance monthly
ms_points = data.wrapper.get_index_points(every="M")
symbol_wrapper = data.get_symbol_wrapper(freq="1D")

# Equal weight allocations
equal_weight_allocations = symbol_wrapper.fill()
for idx in ms_points:
    equal_weight_allocations.iloc[idx] = 0.2  # 20% each

# Alternate allocations (50% MSFT, 12.5% others)
alternate_allocations = symbol_wrapper.fill()
for idx in ms_points:
    alternate_allocations.iloc[idx] = [0.125, 0.5, 0.125, 0.125, 0.125]  # [AAPL, MSFT, GOOGL, AMZN, META]

# Remove rows with NaN values for printing
# equal_weight_allocations = equal_weight_allocations[~equal_weight_allocations.isnull().any(axis=1)]
# alternate_allocations = alternate_allocations[~alternate_allocations.isnull().any(axis=1)]

# print("Equal Weight Allocations:")
# print(equal_weight_allocations)
# print("\nAlternate Allocations:")
# print(alternate_allocations)

# Create two portfolios
pf_equal_weight = vbt.Portfolio.from_orders(
    sma_strategy, 
    size=equal_weight_allocations, 
    size_type='target_percent', 
    group_by=True, 
    init_cash=1000, 
    cash_sharing=True
)

pf_alternate = vbt.Portfolio.from_orders(
    sma_strategy, 
    size=alternate_allocations, 
    size_type='target_percent', 
    group_by=True, 
    init_cash=1000, 
    cash_sharing=True
)

pf_rsi = vbt.Portfolio.from_orders(
    rsi_strategy, 
    size=alternate_allocations, 
    size_type='target_percent', 
    group_by=True, 
    init_cash=1000, 
    cash_sharing=True
)

pf_rsi_alternate = vbt.Portfolio.from_orders(
    rsi_strategy, 
    size=alternate_allocations, 
    size_type='target_percent', 
    group_by=True, 
    init_cash=1000, 
    cash_sharing=True
)

pf_equal_weight_benchmark = vbt.Portfolio.from_orders(
    close,
    size=equal_weight_allocations,
    size_type='target_percent',
    group_by=True,
    init_cash=1000,
    cash_sharing=True
)

pf_alternate_benchmark = vbt.Portfolio.from_orders(
    close,
    size=alternate_allocations,
    size_type='target_percent',
    group_by=True,
    init_cash=1000,
    cash_sharing=True
)



# Plot and show statistics for both portfolios
pf_equal_weight.plot(title='Equal Weight').show()
pf_alternate.plot(title='Alternate').show()
pf_rsi.plot(title='RSI').show()
pf_rsi_alternate.plot(title='RSI Alternate').show()
pf_equal_weight_benchmark.plot(title='Equal Weight Benchmark').show()
pf_alternate_benchmark.plot(title='Alternate Benchmark').show()

pd.concat([
    pf_equal_weight.stats(), 
    pf_alternate.stats(), 
    pf_rsi.stats(),
    pf_rsi_alternate.stats(),
    pf_equal_weight_benchmark.stats(), 
    pf_alternate_benchmark.stats()], axis=1, keys=[
        'Equal Weight', 
        'Alternate', 
        'RSI',
        'RSI Alternate',
        'Equal Weight Benchmark', 
        'Alternate Benchmark'
    ])


,Equal Weight,Alternate,RSI,RSI Alternate,Equal Weight Benchmark,Alternate Benchmark
Start Index,2015-01-02 00:00:00-05:00,2015-01-02 00:00:00-05:00,2015-01-02 00:00:00-05:00,2015-01-02 00:00:00-05:00,2015-01-02 00:00:00-05:00,2015-01-02 00:00:00-05:00
End Index,2023-12-29 00:00:00-05:00,2023-12-29 00:00:00-05:00,2023-12-29 00:00:00-05:00,2023-12-29 00:00:00-05:00,2023-12-29 00:00:00-05:00,2023-12-29 00:00:00-05:00
Total Duration,2264 days 00:00:00,2264 days 00:00:00,2264 days 00:00:00,2264 days 00:00:00,2264 days 00:00:00,2264 days 00:00:00
Start Value,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
Min Value,985.795446,988.675255,1000.0,1000.0,1000.0,1000.0
Max Value,4712.917681,5394.648635,2528.684881,2528.684881,7845.814429,8880.500992
End Value,4685.155536,5378.468208,2208.770275,2208.770275,7799.597489,8853.865282
Total Return [%],368.515554,437.846821,120.877027,120.877027,679.959749,785.386528
Benchmark Return [%],354.745304,354.745304,119.872891,119.872891,636.045024,636.045024
Position Coverage [%],99.116608,99.116608,99.116608,99.116608,99.116608,99.116608


For a gut check you can always look at the orders

In [233]:
pf_equal_weight.orders.records_readable.sort_values(by='Index')
# order by 'Index'

,Order Id,Column,Index,Size,Price,Fees,Side
0,0,"(AAPL, AAPL)",2015-02-02 00:00:00-05:00,0.200000,1000.000000,0.0,Buy
335,0,"(META, META)",2015-02-02 00:00:00-05:00,0.200000,1000.000000,0.0,Buy
162,0,"(GOOGL, GOOGL)",2015-02-02 00:00:00-05:00,0.200000,1000.000000,0.0,Buy
248,0,"(AMZN, AMZN)",2015-02-02 00:00:00-05:00,0.200000,1000.000000,0.0,Buy
81,0,"(MSFT, MSFT)",2015-02-02 00:00:00-05:00,0.200000,1000.000000,0.0,Buy
...,...,...,...,...,...,...,...
80,80,"(AAPL, AAPL)",2023-12-01 00:00:00-05:00,0.004882,5569.060236,0.0,Sell
247,85,"(GOOGL, GOOGL)",2023-12-01 00:00:00-05:00,0.014743,2591.853256,0.0,Buy
334,86,"(AMZN, AMZN)",2023-12-01 00:00:00-05:00,0.001059,3857.567895,0.0,Sell
161,80,"(MSFT, MSFT)",2023-12-01 00:00:00-05:00,0.002054,6578.282527,0.0,Sell


# Use Regime Filter for Weights
Let's look at the slope of the moving average of the SPY to set our regime. 

We will then create a dictionary of portfolio weights for each regime. Then apply that to our portfolio.

In [258]:
def regime_detection(spy, timeperiod=50):
    spy_sma = spy.run('sma', timeperiod=timeperiod).sma
    # Bullish when sma is increasing
    bullish = np.full_like(spy_sma.values, False) # create an array of False values
    bullish[1:] = spy_sma.values[1:] > spy_sma.values[:-1] # set the values to True if the sma is increasing
    return pd.Series(bullish, index=spy_sma.index) # return a DataFrame with the regime

regime = regime_detection(spy)

# Plot the regime
spy.close.vbt.overlay_with_heatmap(regime).show()
    

Let's have a look at how each strategy performs

In [259]:
# Plot the various strategies
strategy.vbt.rebase(100).vbt.plot().show()
# Plot the regime
spy.close.vbt.overlay_with_heatmap(regime).show()


In [260]:
strategy.columns


Index(['AAPL_sma', 'MSFT_sma', 'GOOGL_sma', 'AMZN_sma', 'META_sma', 'AAPL_rsi',
       'MSFT_rsi', 'GOOGL_rsi', 'AMZN_rsi', 'META_rsi'],
      dtype='object', name='symbol')

In [277]:
# Create a dict that maps the we allocate more to SMA and less to RSI in a bullish market and vice versa in a bearish market
regime_dict = {
    'Bullish': {
        'AAPL_sma': 0.30, 'MSFT_sma': 0.30, 'GOOGL_sma': 0.30, 'AMZN_sma': 0.30, 'META_sma': 0.30,
        'AAPL_rsi': 0.05, 'MSFT_rsi': 0.05, 'GOOGL_rsi': 0.05, 'AMZN_rsi': 0.05, 'META_rsi': 0.05
    }, # 2x SMA, 0.5x RSI in bullish market
    'Bearish': {
        'AAPL_sma': 0.05, 'MSFT_sma': 0.05, 'GOOGL_sma': 0.05, 'AMZN_sma': 0.05, 'META_sma': 0.05,
        'AAPL_rsi': 0.30, 'MSFT_rsi': 0.30, 'GOOGL_rsi': 0.30, 'AMZN_rsi': 0.30, 'META_rsi': 0.30
    } # 0.5x SMA, 2x RSI in bearish market
}

def get_allocations(regime_value):
    if regime_value:
        return regime_dict['Bullish']
    else:
        return regime_dict['Bearish']

# Apply the function to each element of the regime Series
allocations = regime.apply(get_allocations)

# Convert the result to a DataFrame
allocations = pd.DataFrame(allocations.tolist(), index=regime.index)

# Print the allocations
# print(allocations)

pf_w_regime = vbt.Portfolio.from_orders(
    strategy,
    size=allocations,
    size_type='target_percent',
    init_cash=1000,
    cash_sharing=True,
    leverage= 1.75  # Total allocation sums to 1.75 (175%)
)

pf_w_regime.plot(title='Regime-based Portfolio').show()

In [280]:
pf_w_regime['2015'].plot_allocations(height=500).show()